# Exercise 15: Power Analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below. 

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill. 

`c` is reading comprehension, and `x` is word reading experience.

In [26]:
options(warn = -1)

sample_size = 100 # How many children in data set? 
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0. 
sd_c = 85      # standard dev of gaussian noise term, epsilon_c

simulate_data <- function(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
  age = runif(sample_size, age_lo, age_hi) # generating age (months)
  x = beta_xa*age + beta_x0 + rnorm(sample_size, sd = sd_x) # word reading experience, generate noise terms
  c = beta_ca*age + beta_cx*x + beta_c0 + rnorm(sample_size, sd = sd_c) # reading comprehension, generate noise terms
  return(data.frame(age = age, read.exp = x, read.comp = c)) # it's actually bad form to have a variable named "c" in R, my bad...
}
# need to rename my "x" and my "c" as with Exercise 14 so x = read.exp, c = read.comp

dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)

,age,read.exp,read.comp
,<dbl>,<dbl>,<dbl>
1,160.9845,166.05327,604.13740
2,122.2649,140.82085,602.84765
3,177.0824,68.66255,221.76789
4,143.3274,-10.40299,96.79726
5,160.8234,189.16725,727.24148
6,123.4835,-50.69627,98.54325


---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data. 

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously. 

In [19]:
# install and load packages here; takes FOREVER, so I split up the code chunks
install.packages("mediation")
library(mediation)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [27]:
## COPIED FROM HW 14; sanity check that this works as before

# setting seed
set.seed(123)

# linear regression part I
  # Age (X) & Word Reading Experience (M)
model1 <- lm(read.exp ~ age, data = dat)
summary(model1)

# linear regression part II
  # Reading Comprehension (Y), Word Reading Experience (M), & Age (X1)
model2 <- lm(read.comp ~ read.exp + age, data = dat)
summary(model2)

# mediation function; see tutorial for help
model3 <- mediate(model1, model2, treat = "age", mediator = "read.exp")
summary(model3)


Call:
lm(formula = read.exp ~ age, data = dat)

Residuals:
     Min       1Q   Median       3Q      Max 
-117.536  -35.510   -0.338   28.659  166.822 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)  
(Intercept)  41.6312    22.3442   1.863   0.0654 .
age           0.1944     0.1538   1.265   0.2090  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 52.91 on 98 degrees of freedom
Multiple R-squared:  0.01606,	Adjusted R-squared:  0.006017 
F-statistic: 1.599 on 1 and 98 DF,  p-value: 0.209



Call:
lm(formula = read.comp ~ read.exp + age, data = dat)

Residuals:
    Min      1Q  Median      3Q     Max 
-147.01  -59.30  -13.12   55.16  188.17 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -10.7081    35.5521  -0.301  0.76391    
read.exp      3.0356     0.1580  19.218  < 2e-16 ***
age           0.9218     0.2424   3.803  0.00025 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 82.73 on 97 degrees of freedom
Multiple R-squared:  0.8083,	Adjusted R-squared:  0.8043 
F-statistic: 204.4 on 2 and 97 DF,  p-value: < 2.2e-16



Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

               Estimate 95% CI Lower 95% CI Upper p-value   
ACME              0.575       -0.340         1.49   0.202   
ADE               0.926        0.457         1.41   0.002 **
Total Effect      1.501        0.506         2.54   0.002 **
Prop. Mediated    0.382       -0.572         0.67   0.200   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Sample Size Used: 100 


Simulations: 1000 


In [28]:
# run_analysis() function
run_analysis <- function(data) {
  runif(sample_size, age_lo, age_hi)
  model1 <- lm(read.exp ~ age, data = data)
  model2 <- lm(read.comp ~ read.exp + age, data = data)
  model3 <- mediate(model1, model2, treat = "age", mediator = "read.exp")
  d0 <- model3$d0 ## ACME estimate
  d0.p <- model3$d0.p ## ACME p-value
  z0 <- model3$z0 ## ADE estimate
  z0.p <- model3$z0.p # ADE p-value
  return(c(d0, d0.p, z0, z0.p))
}

---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [29]:
# repeat_analysis() function
repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi, 
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {

    # Simouts matrix for storing each output from my run_analysis() function
    simouts <- matrix(rep(NA, num_simulations*4), nrow=num_simulations, ncol=4)

    # Start simulating; YAY another for loop
    for (i in 1:num_simulations) {
      data.sim <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
      simouts[i,] <- run_analysis(data.sim)}
      
    # Calculate coverage for both ACME and ADE estimates using generated p-values
    ACME_cov = mean(simouts[,2]<=alpha) 
    ADE_cov = mean(simouts[,4]<=alpha)
    return(list(ACME_cov = ACME_cov, ADE_cov = ADE_cov))}

Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01. 

In [30]:
# repeat_analysis() function; 10 simulations, alpha criterion of .01
  # takes a little while to run the simulations
repeat_analysis(num_simulations = 10, alpha = .01, sample_size, age_lo, age_hi, 
                beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)

$ACME_cov
[1] 0.8

$ADE_cov
[1] 0.7

---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run. 

In [32]:
# varying sample sizes; takes quite a while to run this for loop
var_sample_size <- c(50, 75, 100, 125, 150)
num_sample_size <- length(var_sample_size)
coverage_loop <- matrix(NA, ncol = 3, nrow = 5)
colnames(coverage_loop) <- c("num_sim", "ACME_cov", "ADE_cov")

# using another for loop woo hoo
for (i in 1:num_sample_size) {
  sample_size = var_sample_size[i]
  coverage_loop[i,"num_sim"] <- sample_size
  
  # cov_values
  cov_values <- repeat_analysis(num_simulations = 10, alpha = 0.01, sample_size = sample_size, 
                  age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
  
  # coverage loop
  coverage_loop[i, "ACME_cov"] <- cov_values$ACME_cov
  coverage_loop[i, "ADE_cov"] <- cov_values$ADE_cov}

Print your results. 

In [33]:
# print my results from my coverage loop
print(coverage_loop)

     num_sim ACME_cov ADE_cov
[1,]      50      0.5     0.4
[2,]      75      0.6     0.7
[3,]     100      0.7     0.5
[4,]     125      1.0     0.7
[5,]     150      1.0     0.7


## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above? 

> *As we increase our sample size , our coverage for our ACME (ACME_cov) and our ADE (ADE_cov) both increase.* 

**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> *Write your response here.* 

> *Our Word Reading Experience variable, was created by adding a beta coefficient (a one unit/month increase in Age results in a __ unit change in Word Reading Experience) and an intercept coefficient (when Age = 0, what one's Word Reading Experience is __). Our generated data is created to be normally distributed numbers and have generated noise terms. Word Reading Experience (M) is scaled with Age by 0.5 (see simulated data at the beginning of this exercise). This therefore makes it harder to detect the direct effect of Age on Reading Comprehension compared to our mediating effect.*

**DUE:** 5pm EST, April 5, 2023

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> *Someone's Name*